In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('cleaned_traindata.csv')
train.head()

,Unnamed: 0,question_text,target
0,0,quebec nationalist see province nation 1960,0
1,1,adopt dog would encourage people adopt not shop,0
2,2,velocity affect time velocity affect space geo...,0
3,3,otto von guericke use magdeburg hemisphere,0
4,4,convert montra helicon mountain bike change tyre,0


In [3]:
from sklearn.model_selection import train_test_split

xytrain, xyval = train_test_split(train, test_size=0.2)

In [4]:
XYTrain = xytrain.applymap(str)
XYValid = xyval.applymap(str)

In [8]:
X_Train = XYTrain['question_text'].values
Y_Train = XYTrain['target'].values
X_Valid = XYValid['question_text'].values
Y_Valid = XYValid['target'].values
print(X_Train.shape)
print(Y_Train.shape)

(1044897,)
(1044897,)


In [9]:
Y_Train

array(['0', '0', '0', ..., '1', '0', '0'], dtype=object)

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 20000
seq_length = 100
embedding_dim = 300
 
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(list(X_Train))
token_train = tokenizer.texts_to_sequences(X_Train)
token_valid = tokenizer.texts_to_sequences(X_Valid)
xtrain = pad_sequences(token_train, maxlen = seq_length)
xvalid = pad_sequences(token_valid, maxlen = seq_length)

Using TensorFlow backend.


In [15]:
from keras.layers import *
from keras.models import Model
from keras.optimizers import *

def get_model(lr=0.005):
    inputs = Input(shape=(seq_length,))
    x = Embedding(num_words, embedding_dim)(inputs)
    x = SpatialDropout1D(0.1)(x)
    x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outputs = Dense(64, activation="relu")(conc)
    outputs = Dense(1, activation='sigmoid')(outputs)
    model = Model(inputs=inputs, outputs=outputs)
    optimizer = Adam(lr=lr)
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    print(model.summary())
    return model

NameError: name 'model2' is not defined

In [21]:
import tqdm
model = get_model()
tqdm(model.fit(xtrain,Y_Train,batch_size=512,epochs=2, verbose = 1))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 100, 300)     6000000     input_7[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_6 (SpatialDro (None, 100, 300)     0           embedding_6[0][0]                
__________________________________________________________________________________________________
lstm_layer (LSTM)               (None, 100, 60)      86640       spatial_dropout1d_6[0][0]        
__________________________________________________________________________________________________
global_ave

TypeError: 'module' object is not callable

In [22]:
model.save('model-v1.h5')

In [23]:
import pickle
with open('tokenizer.txt','wb+') as f:
    pickle.dump(tokenizer,f)
    f.close()

In [25]:
import keras
model = keras.models.load_model('model-v1.h5')

In [26]:
pred = model.predict(xvalid)

In [35]:
binary = np.round(pred)
Y_Valid = [float(i) for i in Y_Valid]

In [36]:
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score,roc_curve,auc

print(confusion_matrix(Y_Valid,binary))
print(classification_report(Y_Valid, binary))

[[241687   3297]
 [  8792   7449]]
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98    244984
         1.0       0.69      0.46      0.55     16241

   micro avg       0.95      0.95      0.95    261225
   macro avg       0.83      0.72      0.76    261225
weighted avg       0.95      0.95      0.95    261225

